In [1]:
# define a state class such that it can be used to measure some or all qubits, evolve a random brick work circuit
# compute expectation value of a given Pauli string
# apply a specific gate on it, could be supplied as a full unitary or 
# state attributes : norm, full array
# state.measure([Z Z Z])
# state.bwm_evolve(n_steps):
# state.apply(gate)
# state.reset()
# state.compute_ee( ) # entanglement entropy

In [89]:
import numpy as np
from itertools import product

X=np.array([[0,1],[1,0]],dtype=complex)
Y=np.array([[0,-1j],[1j,0]],dtype=complex)
Z=np.array([[1,0],[0,-1]],dtype=complex)

class qstate:
    def __init__(self,n=1,random=False,arr=None):
        
        self.nqubits=n
        
        if(random==True):
            self.arr=np.random.rand(2**n)+1j*np.random.rand(2**n)
            self.arr=self.arr/np.linalg.norm(self.arr)
            
        elif(random==False and not arr):
            self.arr=np.zeros(2**n,dtype=complex)
            self.arr[0]=1
            
        elif(arr):
            self.arr=arr/np.linalg.norm(arr)
            self.nqubits=int(np.log2(len(arr)))
            
        self.norm=np.linalg.norm(self.arr)
    
    
    def apply_1gate(self,gate,i=1,inplace=True):

        #apply gate on qubit i
        
        Ntot=2**self.nqubits # total dimesnions of the Hilbert space

        arr=self.arr.copy()
    
        # say if m ranges from 0 to Ntot-1, mp ranges from 0 to Ntot/2 -1 since ith position bit string is fixed

        L=self.nqubits
        
        alpha,beta=0,1 # two states which can be generalized for qudits later
        
        for mp in range(Ntot//2):

            # suppose mp = m1'+m2', where m1'=b1x2**(L-2)+b2x2**(L-3)+... +b_{i-1}x2**(L-i), m2'=b_{i+1}2**(L-i-1)+...+b_{L-1}2**(1)+b_{L}q**(0)
            # then by definition m1' = (2**(L-i))*mp//2**(L-i)
            # and m2'=mp-m1'
            m1p=(2**(L-i))*(mp//(2**(L-i)))
            m2p=mp-m1p

            # now the two states which will be modified are ones which have alpha and beta at ith position

            alpha_ind=2*m1p+alpha*(2**(L-i))+m2p

            beta_ind=2*m1p+beta*(2**(L-i))+m2p

            # modifying the state at the corresponding indices

            arr[alpha_ind]=self.arr[alpha_ind]*gate[0,0]+gate[0,1]*self.arr[beta_ind]
            arr[beta_ind]=self.arr[beta_ind]*gate[1,1]+gate[1,0]*self.arr[alpha_ind]
        if(inplace==True):
            self.arr=arr
            self.norm=np.linalg.norm(self.arr)
            return None
        else:
            return arr
    
    #def compute_ee(self,)
        
    def measure(self,basis_list=['Z'],inplace=False,only_subsystem=False):
        

        # collapse the state by applying Pauli gates and then measuring only the qubits specified
        if(len(basis_list)!=self.nqubits):
            basis_list=['Z']*self.nqubits
        meas_qubits=[]
        for i in range(self.nqubits):
            
            if(basis_list[i]=='X'):
                self.arr.apply_1gate(X,i)
                meas_qubits.append(i)
            elif(basis_list[i]=='Y'):
                self.arr.apply_1gate(Y,i)
                meas_qubits.append(i+1)
                
            elif(basis_list[i]=='Z'):
                meas_qubits.append(i)
                
        all_qubits=[i for i in range(self.nqubits)]
        
        
        shape_arr=np.repeat(2,self.nqubits)
        La=len(meas_qubits)
        print("all,meas",all_qubits,meas_qubits)
        traced_qubits=np.setdiff1d(all_qubits,meas_qubits)
        
        print("traced",traced_qubits)
        psi=np.reshape(self.arr,shape_arr)

        if(La!=self.nqubits):
            prob_arr=np.sum(np.abs(psi)**2,axis=tuple(traced_qubits)) #tracing out qubits that won't be measured
        
        else:
            prob_arr=np.abs(psi)**2
        prob_arr=np.reshape(prob_arr,2**La)

        meas_int=np.random.choice(range(2**La),p=prob_arr) #choosing the computational basis state randomly after measurement based on 
        print("measured int",meas_int)
        
        # collapsed state on the unmeasured system
        if(only_subsystem):
            psi_b=np.zeros(np.repeat(2,self.nqubits-La),dtype="complex")

        else:
            psi_b=np.zeros(np.repeat(2,self.nqubits),dtype="complex")
            
        binary = format(meas_int,'0'+str(La)+'b')
        
        bin_ind=list(int(binary[i]) for i in range(La))

        full_ind=np.repeat(0,self.nqubits)
        full_ind[meas_qubits]=bin_ind
        print("full ind starting as ",full_ind)
        
        if(La==self.nqubits):
            psi_b[full_ind]=psi[full_ind]
            
        for c in product(range(2),repeat=self.nqubits-La):
            print("indices",list(c),full_ind)
            for i,ind in enumerate(traced_qubits):
                full_ind[ind]=c[i]
            print("shapde check",full_ind,np.shape(psi),np.shape(psi_b),psi[1,0])
            if(only_subsystem):
                psi_b[list(c)]=psi[full_ind]
            else:
                psi_b[full_ind]=psi[full_ind]
                
        psi_b=psi_b/np.linalg.norm(psi_b)
        if(only_subsystem):
            psi_b=np.reshape(psi_b,2**(self.nqubits-La))
            self.nqubits=self.nqubits-La
            
        else:
            psi_b=np.reshape(psi_b,2**(self.nqubits))
            self.nqubits=self.nqubits                          
                             
        self.arr=psi_b/np.linalg.norm(psi_b)
        
        return bin_ind

In [90]:
state1=qstate(arr=[0.0,0.3,0.6,0.0])
print(state1.arr)

X=np.array([[0,1],[1,0]])
state1.apply_1gate(X,inplace=False)
print('state after')
print(state1.arr)
state1.measure(['Z','I'],only_subsystem=True)
print("state after collapse",state1.arr)

[0.         0.4472136  0.89442719 0.        ]
state after
[0.         0.4472136  0.89442719 0.        ]
all,meas [0, 1] [0]
traced [1]
measured int 1
full ind starting as  [1 0]
indices [0] [1 0]
shapde check [1 0] (2, 2) (2,) 0.8944271909999159


ValueError: shape mismatch: value array of shape (2,2) could not be broadcast to indexing result of shape (1,)